In [1]:
import pandas as pd; import numpy as np; import os;import random;import time;

In [2]:
import random; import gzip;

# Explorar geo datos de llamados (Voice)
en un principio si los datos no son geolocalizados por antena no puedo saber quien llamo a la zona endemica. El objetivo es ver...

* como se compone la base de datos
* que sirve para el analisis de la mobilidad y del "Gran Chaco"
* haria falta tambien tener el .cpp que asigna home a cada user


In [3]:
np.random.seed(2015)

In [10]:
##setting general params
rootdir="/home/juan/mobility-study"
os.chdir(rootdir)
directory = "0315"; contract_type="Prepago";
day_streak="03-04"
input_file= 'datosgeo/%s/%s/Geo_Voz_%s_%s%s.txt.gz' % (directory,contract_type,contract_type,day_streak,directory)

In [16]:
input_file

'datosgeo/0315/Prepago/Geo_Voz_Prepago_03-040315.txt.gz'

In [11]:
!ls datosgeo/0315/Prepago/

Geo_Voz_Prepago_01-020315.txt.gz  Geo_Voz_Prepago_17-180315.txt.gz
Geo_Voz_Prepago_03-040315.txt.gz  Geo_Voz_Prepago_19-200315.txt.gz
Geo_Voz_Prepago_05-060315.txt.gz  Geo_Voz_Prepago_21-220315.txt.gz
Geo_Voz_Prepago_09-100315.txt.gz  Geo_Voz_Prepago_23-230315.txt.gz
Geo_Voz_Prepago_11-120315.txt.gz  Geo_Voz_Prepago_25-260315.txt.gz
Geo_Voz_Prepago_13-140315.txt.gz  Geo_Voz_Prepago_27-280315.txt.gz
Geo_Voz_Prepago_15-160315.txt.gz  Geo_Voz_Prepago_29-310315.txt.gz


In [14]:
!zcat $input_file | head - n10

==> standard input <==
CONTRATO	TELEFONO	ORIGEN	DESTINO	DIRECCION	DURACION_SEGUNDOS	UBICACIONCIUDAD	LATITUD	LONGITUD	OPERADOR	FECHA_LLAMADA
	00000B984B91C296144E8434BB08D508	00000B984B91C296144E8434BB08D508	5E6693E9FBF7E6F541DE2D17A771E443	ENTRANTE	152				IUSACELL          	03/03/2015 05:55:15 a.m.
	00000C5D3BFBEBD919F6F68386C3DD5E	00000C5D3BFBEBD919F6F68386C3DD5E	0A8D39BFACCA596573FA1371E02A4EF2	ENTRANTE	116		15.85830556	-97.04511111	IUSACELL          	04/03/2015 08:16:16 a.m.
	000011F2BE4C5E7D456026CD791185B3	000011F2BE4C5E7D456026CD791185B3	00F0D512BCD00EC7B9C666ECA0B645C3	ENTRANTE	19	Mexicali	32.574342	-115.352511	UNEFON            	04/03/2015 06:05:47 p.m.
	00001228A0D38FB8D81365EDAB963130	00001228A0D38FB8D81365EDAB963130	106BD5A02DE8DADBE6DAF2E82EED04DE	ENTRANTE	187		20.73645278	-103.4314972	UNEFON            	03/03/2015 05:05:32 p.m.
	0000129F4191FFD0963774D2307C7510	0000129F4191FFD0963774D2307C7510	B55022400C295828668E644CFDA58425	ENTRANTE	46	Tehuacan	18.45369522	-97.40105726	U

In [33]:
pd.read_csv?

In [106]:
tabla_mensual = pd.read_csv(
                input_file,
                delim_whitespace=True,
                #sep = "\t",
                names = ['Telefono','Target','Destination','Direction','Duration',
                         'City','Latitude','Longitude','Operator','Date1','Date2','Date3'],
                engine = 'c', lineterminator='\n', skipinitialspace=True,
                index_col=False,
                skiprows = 2,
                #usecols = ['Latitude','Longitude']
    #dtype = {'Target':np.uint32,'Destination':np.uint32,'TimeStamp':np.uint32,
     #                                 'Duration':np.uint16,'AntennaID':np.uint16,'Direction':np.object_}
                )

In [107]:
tabla_mensual.drop(['Telefono','Direction','Duration','Operator','Date1','Date2'],inplace=True,axis=1)

In [108]:
#tratamos de castear los valores de una series a cosas numericas y sino devolvemos False
def try_int_cast(x):
    try: 
        float(x)
        return True
    except:
        return False

In [109]:
buff = tabla_mensual.ix[tabla_mensual['Date3'].isnull()]
#buscamos todas aquellas filas que no podemos castear a 
buff2 = buff[~buff['City'].apply(lambda x: try_int_cast(x))]

In [110]:
tabla_mensual.drop(buff2.index.values,inplace=True,axis=0)

In [118]:
#guardo los valores a reemplazar
buff = tabla_mensual.ix[tabla_mensual['Date3'].isnull(),['City','Latitude']]

In [121]:
#hago el reemplazo en ese indice
indice = tabla_mensual.ix[tabla_mensual['Date3'].isnull()].index.values

In [129]:
#el .values es crucial. Si no esta, va a tratar de igualar los valores basandose en indice. En cambio
#el .values tira los indices y lo ve com una "matriz" pura sin labels.
tabla_mensual.loc[indice,['Latitude','Longitude']]= buff.values

In [125]:
buff.sample(100)

,City,Latitude
4307647,18.45705867,-95.35011002
2175989,18.64722222,-98.605
8792634,20.5478,-100.8228
4878816,19.26988757,-99.65918728
5971496,19.085767,-98.199828
790427,20.096808,-98.751367
8937487,18.54232306,-92.65082833
4004321,14.68219444,-92.15552778
3213375,25.67732778,-100.3189194
5684929,20.50982306,-87.23321139


In [130]:
tabla_mensual.sample(100)

,Target,Destination,City,Latitude,Longitude,Date3
3714794,5C28E3B10F2E5C377BA61A4095700418,C89D73B52F04CF3A18B56E15396D7BF2,Del,Miguel,Hidalgo,04/03/2015
1208769,1DD85CC71801A1B48836C01C747FCE54,81A07EB44B8820AB281C99FD8FD2BBD9,Del,Tlahuac,19.295068,05:45:58
5652957,8C3CCB4D8CA3AD3BB481E7BD12E3F727,A41D9AC8E356B84EEC8E24E0F40E27E6,20.023511,20.023511,-98.8413,NaN
7806954,C190852CF77278E8EB3F95391FF1EDA0,54A28B1DA237B0EBAA2A17C4DD4EB70C,Salina,Cruz,16.1897,01:20:41
6138282,983ACB6546F8F6BB3F25BC1CF1350A28,A417C47E2BAF014220894E39A233394D,Chimalhuacan,19.3865,-98.914194,p.m.
9955551,F6B598A8AA63C8EA4435CABC0C1E6B1B,8F6A45BC5519F981968A47C9DC634AE0,Del,Iztacalco,19.38847,10:36:46
10028510,F881B75551261C0E7ECAB1E9B9630136,B830CDFF47BD68330491B0162A2FBA8A,19.42677799,19.42677799,-99.10868169,NaN
2339455,39F62DBA070C39B884065AB2DC895FD3,3C3AD6AF30989DF88339A0EE0DDF5C95,18.95805556,18.95805556,-98.23888889,NaN
1169213,1CDFA83FB2AD2A6AF6E16FE23277A53E,6C22E3691A17816F4A5E7134996BF077,Del,Venustiano,Carranza,04/03/2015
6912701,AB5CCB9B9DB6674149A499BF839BF853,5FCF8AF473C828D4A9B50B2B4064E62A,17.46747222,17.46747222,-97.21580556,NaN


In [78]:
try: 
    int(buff2.loc[10328051].City)
except""   

'UNEFON'

In [64]:
tabla_mensual.head()

,Target,Destination,City,Latitude,Longitude,Date3
0,00000C5D3BFBEBD919F6F68386C3DD5E,0A8D39BFACCA596573FA1371E02A4EF2,15.85830556,-97.04511111,NaN,NaN
1,000011F2BE4C5E7D456026CD791185B3,00F0D512BCD00EC7B9C666ECA0B645C3,Mexicali,32.574342,-115.352511,p.m.
2,00001228A0D38FB8D81365EDAB963130,106BD5A02DE8DADBE6DAF2E82EED04DE,20.73645278,-103.4314972,NaN,NaN
3,0000129F4191FFD0963774D2307C7510,B55022400C295828668E644CFDA58425,Tehuacan,18.45369522,-97.40105726,p.m.
4,0000129F4191FFD0963774D2307C7510,D3E9BDC02EF826B7BB9E01B6F3EC00BC,18.4889,-97.3836,NaN,NaN


In [50]:
tabla_mensual.columns[5:-1]

Index([u'City', u'Latitude', u'Longitude', u'Operator', u'Date1', u'Date2'], dtype='object')

,Telefono,Target,Destination,Direction,Duration,City,Latitude,Longitude,Operator,Date1,Date2,Date3
0,00000C5D3BFBEBD919F6F68386C3DD5E,00000C5D3BFBEBD919F6F68386C3DD5E,0A8D39BFACCA596573FA1371E02A4EF2,ENTRANTE,116,15.85830556,-97.04511111,NaN,04/03/2015,08:16:16,a.m.,NaN
1,000011F2BE4C5E7D456026CD791185B3,000011F2BE4C5E7D456026CD791185B3,00F0D512BCD00EC7B9C666ECA0B645C3,ENTRANTE,19,Mexicali,32.574342,-115.352511,UNEFON,04/03/2015,06:05:47,p.m.
2,00001228A0D38FB8D81365EDAB963130,00001228A0D38FB8D81365EDAB963130,106BD5A02DE8DADBE6DAF2E82EED04DE,ENTRANTE,187,20.73645278,-103.4314972,NaN,03/03/2015,05:05:32,p.m.,NaN
3,0000129F4191FFD0963774D2307C7510,0000129F4191FFD0963774D2307C7510,B55022400C295828668E644CFDA58425,ENTRANTE,46,Tehuacan,18.45369522,-97.40105726,UNEFON,04/03/2015,02:08:59,p.m.
4,0000129F4191FFD0963774D2307C7510,0000129F4191FFD0963774D2307C7510,D3E9BDC02EF826B7BB9E01B6F3EC00BC,ENTRANTE,11,18.4889,-97.3836,NaN,03/03/2015,01:09:30,p.m.,NaN


In [10]:
#para ver valores unicos lo transformamos a un conjunto
set(tabla.OPERADOR.values)

{nan,
 'ALESTRA           ',
 'AVANTEL           ',
 'AXTEL             ',
 'BESTEL            ',
 'BESTPHONE         ',
 'CABLECOM          ',
 'CABLEMAS          ',
 'IP MATRIX         ',
 'IUSACELL          ',
 'IUSATEL           ',
 'MARCATELCOM       ',
 'MAXCOM            ',
 'MEGA CABLE        ',
 'MEGACABLE (MCM)   ',
 'METRORED          ',
 'MOVISTAR          ',
 'NO IDENTIFICADO   ',
 'OPCOM             ',
 'OPERBES           ',
 'PROTEL I-NEXT     ',
 'PROTELOP          ',
 'TELCEL            ',
 'TELECABLE         ',
 'TELEFONICA        ',
 'TELMEX            ',
 'TELNOR            ',
 'TOTAL PLAY        ',
 'UNEFON            '}

## Obs.
* estan cambiadas las columnas ORIGEN y DESTINO 
* CONTRATO es todo NaNs
* OPERADORA y DURACION_SEGUNDOS no aportan info, 

In [14]:
tabla.columns

Index([u'TELEFONO', u'DESTINO', u'ORIGEN', u'DIRECCION', u'UBICACIONCIUDAD',
       u'LATITUD', u'LONGITUD', u'OPERADOR', u'FECHA_LLAMADA'],
      dtype='object')

In [14]:
tabla.shape

(7860676, 7)

In [10]:
tabla.rename(columns={'ORIGEN': 'DESTINO', 'DESTINO': 'ORIGEN'},inplace=True);
#aunque suene extranyo tambien voy a tirar la columna ubicacion/ciudad porque tiene muchos NaNs y 
#porque solo miro la info a nivel 'departamental'
tabla.drop(['CONTRATO','OPERADOR','DURACION_SEGUNDOS','UBICACIONCIUDAD'],axis=1,inplace=True)


In [13]:
#chequeo si hay info faltante
tabla[pd.isnull(tabla).any(axis=1)]
#resulta que hay como 600k de casi 8M

,TELEFONO,DESTINO,ORIGEN,DIRECCION,LATITUD,LONGITUD,FECHA_LLAMADA
26,0000A5ECB320A23C19A91C515A51F04D,0000A5ECB320A23C19A91C515A51F04D,C2430855E8747E6F08EB709DE21FED58,ENTRANTE,NaN,NaN,03/01/2015 08:09:34 a.m.
27,0000A5ECB320A23C19A91C515A51F04D,0000A5ECB320A23C19A91C515A51F04D,C2430855E8747E6F08EB709DE21FED58,ENTRANTE,NaN,NaN,03/01/2015 06:06:42 a.m.
28,0000A5ECB320A23C19A91C515A51F04D,0000A5ECB320A23C19A91C515A51F04D,F687EB84F1786572AF0CC7C4BA1047D8,ENTRANTE,NaN,NaN,03/01/2015 06:03:35 a.m.
353,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,57A17524BF266EEAA7AA425BDB58EC23,ENTRANTE,NaN,NaN,03/01/2015 06:42:18 p.m.
354,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 05:19:30 p.m.
355,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 09:15:42 p.m.
356,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 09:16:47 p.m.
357,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,78F6B0001B8E7F3DCCFC573385A2F54D,ENTRANTE,NaN,NaN,04/01/2015 09:15:07 p.m.
358,0001A6980837058FD3819ED9236C4682,0001A6980837058FD3819ED9236C4682,B1475C3F83AA1CC6E5969CED45F6EBE9,ENTRANTE,NaN,NaN,03/01/2015 07:04:33 p.m.
362,0001B96F3892319D191BCEEAE0AD48DD,0001B96F3892319D191BCEEAE0AD48DD,70C00DE41AB64A5720E59BDB6BE0BC05,ENTRANTE,NaN,NaN,04/01/2015 10:04:14 a.m.


In [16]:
tabla.dropna(inplace=True)

In [17]:
tabla2 = pd.read_csv("datosgeo/Prepago/Geo_Voz_Prepago_01-020115.txt.gz",sep="\t")
tabla2.drop(['CONTRATO','OPERADOR','DURACION_SEGUNDOS','UBICACIONCIUDAD'],axis=1,inplace=True)
tabla2.rename(columns={'ORIGEN': 'DESTINO', 'DESTINO': 'ORIGEN'},inplace=True);
tabla2.dropna(inplace=True)

In [18]:
sample=tabla.ix[random.sample(tabla.index, 3*(10**5))]
sample2=tabla2.ix[random.sample(tabla2.index, 3*(10**5))]


In [ ]:
del tabla; del tabla2

In [19]:
sample.index = range(1,len(sample)+1)
sample2.index = range(1,len(sample2)+1)

In [20]:
test2=sample.ix[100].FECHA_LLAMADA
test2[0:-3]+'m'

'04/01/2015 12:21:05 pm'

In [20]:
import time;

#### paso el date a formato epoch

In [21]:
sample['FECHA_LLAMADA'] = sample['FECHA_LLAMADA'].map(lambda t: time.mktime(time.strptime(t[0:-3]+'m', "%d/%m/%Y %I:%M:%S %p")));
sample2['FECHA_LLAMADA'] = sample2['FECHA_LLAMADA'].map(lambda t: time.mktime(time.strptime(t[0:-3]+'m', "%d/%m/%Y %I:%M:%S %p")));

In [22]:
sample.rename(columns={'FECHA_LLAMADA': 'FECHA_LLAMADA_EPOCH'},inplace=True);
sample2.rename(columns={'FECHA_LLAMADA': 'FECHA_LLAMADA_EPOCH'},inplace=True);

In [13]:
#hago un sort por el FECHA_LLAMADA para ver cuando arranca la base de datos 
tabla.index = range(1,len(tabla)+1)

In [16]:
tabla['FECHA_LLAMADA'] = tabla['FECHA_LLAMADA'].map(lambda t: time.mktime(time.strptime(t[0:-3]+'m', "%d/%m/%Y %I:%M:%S %p")));
tabla.rename(columns={'FECHA_LLAMADA': 'FECHA_LLAMADA_EPOCH'},inplace=True);

In [17]:
tabla.sort_values(by='FECHA_LLAMADA_EPOCH',ascending=True,inplace=True)

In [18]:
tabla.head()

,TELEFONO,DESTINO,ORIGEN,DIRECCION,UBICACIONCIUDAD,LATITUD,LONGITUD,FECHA_LLAMADA_EPOCH
5015223,DC1F8BC68A6B6D22A7CBD4C9787BD9DB,DC1F8BC68A6B6D22A7CBD4C9787BD9DB,F1BB45B86CC4540E6194D6A28F177825,SALIENTE,Tlalnepantla,19.517500,-99.102500,1420070400
6984754,CBF4C39BE4C0EAC246FCD5D206C5D2E4,CBF4C39BE4C0EAC246FCD5D206C5D2E4,9877CC267054BD7E5871D9D5627FD507,ENTRANTE,NaN,18.966750,-99.247667,1420070400
10363183,E3595DB849B1CD0FC19187317ECA639F,E3595DB849B1CD0FC19187317ECA639F,139AD14F044D03AEE227024E84A615C6,ENTRANTE,Apan,19.710194,-98.448611,1420070400
9821410,76D12717E57C739DB1C2AE5ADA26BC92,76D12717E57C739DB1C2AE5ADA26BC92,30A61DC885EDDEA18B8AB707E5A84321,ENTRANTE,Guadalajara,20.670161,-103.382536,1420070400
6754731,501EF14B895B19BF0F6034C310EF85E0,501EF14B895B19BF0F6034C310EF85E0,2843788532C31620AF795210D4E8D891,ENTRANTE,Xonacatlan,19.405165,-99.521802,1420070400


# *Obs* 
no veo nada porque aca arranca desde EPOCH... PERO mirando la documentacion del trac resulta que todo esta seteado para arrancar el 01/01/2012 a las 0 horas